In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import miRNET, miRNET_enrichment
Targets = miRNET.Targets(path_to_miRTarBase='./baseData/hsa_miRTarBase.csv')
import warnings
import collections
from matplotlib import pyplot as plt
import random
from scipy.stats import chi2_contingency
import json
import networkx as nx
from requests.exceptions import ConnectionError
import requests
import requests
import io

def reactome_to_pandas(ReactomeRes):

    url = 'https://reactome.org/AnalysisService/download/' + ReactomeRes.token + \
          '/pathways/TOTAL/result.csv'
    res = requests.get(url).content
    reactome_df = pd.read_csv(io.StringIO(res.decode('utf-8')))

    return reactome_df

In [ ]:
####################
#up-regulated mIRNA#
####################

In [3]:
with open('../Data/miRNAs_key_genes/up_miRNAs_key_genes_dict.json') as json_file:
    up_dict = json.load(json_file)

In [4]:
def grep_target_gene_by_paths(miR_names, miR_dict, Reactome_ID):
    target_gene = list()
    pathway_genes = set()
    for miR in miR_names:
        print(miR)
        target_gene = miR_dict[miR]
        enrich_res = miRNET_enrichment.reactome_enrichment(up_dict[miR], species='Homo sapiens')
        enrich_res = miRNET_enrichment.reac_pars(enrich_res)
        reactome_df = reactome_to_pandas(enrich_res)
    
        for ids in Reactome_ID:
            mask = reactome_df.iloc[:,0] == ids
            if sum(mask) == 0:
                continue
            pg = reactome_df.loc[mask,].iloc[0,12]
            pathway_genes.update(pg.split(';'))
            
    miR_target_pathway_dict = dict((key,[]) for key in miR_names)
    for miR in miR_names:
        miR_target_pathway_dict[miR] = set(miR_dict[miR]).intersection(pathway_genes)
    
    return miR_target_pathway_dict

In [6]:
TGF_b_miRs = ['miR-25-3p', 'miR-24-3p', 'miR-30a-5p', 'miR-150-5p', 'let-7d-5p', 'miR-21-5p', 'miR-29b-3p', 'miR-203a-3p', 'miR-155-5p', 'miR-130a-3p']

TGF_b_ids = ['R-HSA-2173793']

TGFb_miR_genes = grep_target_gene_by_paths(TGF_b_miRs, up_dict, TGF_b_ids)

miR-25-3p
miR-24-3p
miR-30a-5p
miR-150-5p
let-7d-5p
miR-21-5p
miR-29b-3p
miR-203a-3p
miR-155-5p
miR-130a-3p


In [7]:
def dict_to_edge_table(dct):
    res_df = pd.DataFrame(columns=['miR', 'target'])
    for key in dct.keys():
        for elm in dct[key]:
            res_df = pd.concat([res_df, pd.DataFrame(list(zip([key], [elm])), columns=['miR', 'target'])])
    return res_df

In [10]:
dict_to_edge_table(TGFb_miR_genes).to_csv('../Data/edge_SMAD_miR_genes.csv',index=False, index_label=False, sep=',')